In [1]:
from Clasificador import Clasificador 
import numpy as np
import math 
from datos import Datos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
import EstrategiaParticionado



In [14]:

def __normalizar_atributo(datos_atributo,esDiscreto):
    if (esDiscreto):
        return datos_atributo
    else:
        mean = np.mean(datos_atributo)
        std = np.std(datos_atributo)
        return list(map(lambda x: (x - mean) / std, datos_atributo))

In [15]:
print(__normalizar_atributo([1,1,2,3,4],False))

[-0.88235294117647078, -0.88235294117647078, -0.14705882352941191, 0.58823529411764697, 1.3235294117647058]


In [6]:
def normalizar_atributo(datos_atributo,esDiscreto):
    if (esDiscreto):
        return datos_atributo
    else:
        mean = np.mean(datos_atributo)
        std = np.std(datos_atributo)
        return list(map(lambda x: (x - mean) / std, datos_atributo))
def comparativa_con_sklearn(datos):
    estrategia = EstrategiaParticionado.ValidacionCruzada(5)
    sklearn_cv = KFold(n_splits=10,shuffle=True)
    for k in [1,3,5,11,21,51]:
            knn_classifier = KNeighborsClassifier(n_neighbors=k)
            clasificador = ClasificadorKNN(k)
            scores = cross_val_score(knn_classifier, datos[:,:-1],datos[:,-1], cv = sklearn_cv)
            errores = clasificador.validacion(estrategia,tic_tac_toe,clasificador)
            print ("Tasa de acierto k = ",k," :",1 - np.mean(errores))
            print ("Tasa de acierto k = ",k," version sklearn :", np.mean(scores))
            
class ClasificadorKNN (Clasificador):
    datos = np.array([])
    k = 0
    def __init__(self,k):
        super(ClasificadorKNN,self).__init__()
        self.k = k
            
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        datos_traspuestos = datosTrain.transpose()
        datos_normalizados = map(normalizar_atributo,
                                 datos_traspuestos,
                                 atributosDiscretos)
        self.datos = np.array(list(datos_normalizados)).transpose()
  
    def __clasifica_dato(self,dato, atributosDiscretos, diccionario):
        def __dist_euclides(dato_entrenamiento):
            def dist_eje (x,y,esDiscreto):
                return int (x != y) if esDiscreto else (x - y) ** 2
            return math.sqrt(sum(map(dist_eje,
                                     dato_entrenamiento,
                                     dato,
                                     atributosDiscretos)))         
        distancias = map(__dist_euclides,self.datos)
        distancias_min = sorted(zip(distancias,self.datos[:,-1]),key = lambda x: x[0])[:self.k]
        vecinos = list(map(lambda x : x[1], distancias_min))
        frecuencias_clases = list(map(vecinos.count,diccionario[-1].values()))
        return frecuencias_clases.index(max(frecuencias_clases))
                    
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        return [ self.__clasifica_dato (dato,atributosDiscretos,diccionario)
                 for dato in datosTest ]

    


In [7]:
estrategia = EstrategiaParticionado.ValidacionCruzada(10)
wine = Datos ("../ficheros/ConjuntosDatos/wine_proc.data", True)
tic_tac_toe = Datos("../ficheros/ConjuntosDatos/tic-tac-toe.data",True)
print(tic_tac_toe.diccionarios[-1])
print(wine.diccionarios[-1].values())
print(len(estrategia.creaParticiones(tic_tac_toe)[0].indicesTrain))
print(len(estrategia.creaParticiones(tic_tac_toe)[0].indicesTest))

{'negative': 0, 'positive': 1}
dict_values([0, 1, 2])
863
95


In [12]:
comparativa_con_sklearn(tic_tac_toe.datos)


Tasa de acierto k =  1  : 1.0
Tasa de acierto k =  1  version sklearn : 0.894404450262
Tasa de acierto k =  3  : 1.0
Tasa de acierto k =  3  version sklearn : 0.855731893543
Tasa de acierto k =  5  : 1.0
Tasa de acierto k =  5  version sklearn : 0.773134816754
Tasa de acierto k =  11  : 0.997905759162
Tasa de acierto k =  11  version sklearn : 0.590346858639
Tasa de acierto k =  21  : 0.996874831327
Tasa de acierto k =  21  version sklearn : 0.709260471204
Tasa de acierto k =  51  : 0.899865061802
Tasa de acierto k =  51  version sklearn : 0.557646160558


In [8]:
comparativa_con_sklearn(wine.datos)

TypeError: map() must have at least two arguments.